In [12]:
#Note: I used R to simulate fake data and export it to a json file. 
#I will attach the R code and the json file.
#There are two machines, each with their own table: a valve and an oven. 
#The valve has two sensors which transmit their measurements at different intervals.
#The oven has three sensors which transmit their measurements at different intervals.
#The numbers probably don't make sense, they're purely for demonstration purposes.

import json
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """Creates a sqlite3 connection with a db file.
    
    Parameters
    ----------
    db_file : str
        the file name of the .db file you want to make a connection to
        
    Rasies
    ------
    TypeError
        If `db_file` is not a string
    """
    
    if type(db_file) != str:
        raise TypeError('`db_file` argument must be a string.')
    
    conn = None
    try:
        conn = sqlite3.connect(db_file, timeout = 5)
        return conn
    except Error as e:
        print(e)
    return conn


def create_table(conn, title, fields): 
    """creates a new table in the sql database
    
    Parameters
    ----------
    conn: sqlite3.connect
        a connection to a sqlite3 database
    title: str
        the name of your new table
    fields: list[str]
        a list or array-like object containing the column names you want the new table to have
        
    Raises
    ------
    TypeError
        if `title` is not a string or the items in `fields` are not all strings
    """
    
    if type(title) != str:
        raise TypeError('`title` must be a string.')
    if not all(isinstance(item,str) for item in fields):
        raise TypeError('`fields` must be a list of strings.')

    try:
        cur = conn.cursor()
        cur.execute('CREATE TABLE IF NOT EXISTS ' + title + '(' + ', '.join(fields) + ');')
        conn.commit()
    except Error as e:
        print(e)
        
def insert_data(conn, table, values, fields = None, verbose = False):
    """append a row of data values to the end of an existing table in the database
    
    If the argument `fields` is not passed in, the length of the list passed as the `values` argument
    must be equal to the number of columns in the table.
    
    Parameters
    ----------
    conn: sqlite3.connect
        a connection to a sqlite3 database
    table: str
        the name of the table you want to insert values into
    values: list
        a list or array-like object of data values you want to insert into the table
    fields: list[str], optional
        a list of column names matching the list of values you're inserting into the table
        None by default. When `fields` = None, the `values` argument has one value for each column in the table 
    verbose: bool, optional
        if True, the function will print the sql statement used to insert the data
        False by default
    
    Raises
    ------
    ValueError
        If `fields` = None and the length of `values` is not equal to the number of columns in the table
        or if length of `values` is greater than the number of columns in the table.
    TypeError
        If `table` is not a string
    """
    
    if len(values) != len(get_fields(conn,table)):
        raise ValueError('If `fields` argument is left out, there must be exatcly one data value for each field in the table.')
    if len(values) > len(get_fields(conn,table)):
        raise ValueError('Can not have more items in `values` than number of columns in table.')
    if type(table) != str:
        raise TypeError('`table` argument must be a string.')

    try:
        cur = conn.cursor()
        sql_insert = 'INSERT INTO ' + table
        if fields is not None:
            sql_insert += ' (' + ', '.join(fields) + ')'
        sql_insert += ' VALUES ' + '(' + str(list(values))[1:len(str(list(values)))-1] + ')'
        cur.execute(sql_insert)
        conn.commit()
    except Error as e:
        print(e)  
    if verbose:
        print(sql_insert)
        
def import_json(conn, file):
    """imports the data from a json file into the database as one or more tables.
    If the tables don't exist, they are created. If they do exist, 
    the data is inserted into the appropriate tables as long as the row isn't a duplicate 
    of a row that already exists in the table.
    
    Parameters
    ----------
    conn: sqlite3.connect
        a connection to a sqlite3 database
    file: str
        the name of the json file containing the data
    
    Raises
    ------
    TypeError
        If `file` is not a string    
    """
    
    if type(file) != str:
        raise TypeError('`file` must be a string.')
    
    try:
        cur = conn.cursor()
        with open(file) as f:
            data = json.load(f)
        for title,table in data.items():
            if title not in get_tables(conn):
                #if the table doesn't exist, create it
                create_table(conn, title, table[0].keys())
            for row in table:
                #whether it exists or not, go through each row and check if that row exists already
                if tuple(row.values()) not in get_data(conn, title):
                    #if it doesn't already exist, add it to the table.
                    insert_data(conn, title, row.values())
        conn.commit()
    except Error as e:
        print(e)
        

def get_tables(conn):
    """retrieves a list of the names of all of the tables in the database
    
    Parameters
    ----------
    conn: sqlite3.connect
        a connection to a sqlite3 database 
    """
    
    tables = None
    try:
        cur = conn.cursor()
        cur.execute('SELECT name from sqlite_master where type= "table";')
        tables = [table[0] for table in cur.fetchall()]
    except Error as e:
        print(e)
    return tables


def get_fields(conn, table):
    """retrieves a list of the column names of a particular table in the database
    
    Parameters
    ----------
    conn: sqlite3.connect
        a connection to a sqlite3 database 
    table: str
        the name of a table in the database
        
    Raises
    ------
    TypeError
        If `table` is not a string
    """
    
    if type(table) != str:
        raise TypeError('`table` argument must be a string.')
    
    fields = None
    try:
        cur = conn.execute('SELECT * FROM ' + table + ';')
        fields = list(map(lambda x: x[0], cur.description))
    except Error as e:
        print(e)        
    return fields


def get_data(conn, table, fields = None, sensor = None, time = None, where = None, verbose = False):
    """queries the database and retrieves the data as requested by the user
    
    If `fields` argument is not passed, data for all columns of the table will be returned
    If `sensor` agrument is not passed, data for all sensors in that table will be returned
    If `time` argument it not passed, data for all time periods will be returned
    If `where` arhgument is not passed, no further filtering will be done to the data before being returned
    
    Parameters
    ----------
    conn: sqlite3.connect
        a connection to a sqlite3 database 
    table: str
        the name of a table in the database
    fields: list[str], optional
        a list of columns you wish to return from the table.
        None by default. When `fields` = None, every column will be returned.
        `fields` can also be a single string object if you wish to return only one column from the table.
    sensor: int, optional
        the sensor id number of the sensor from the table you wish to retrieve data for.
        None by default. When `sensor` = None, data for all sensors is returned.
        `sensor` can also be a list of integers if you wish to retrieve data from more than one, but not all sensors.
    time: list[str], optional
        the start and end time you wish to filter the data by. 
        Only data values from after the start time and before the end time will be returned (inclusive).
        None by default. When `time` = None, data for all time values will be returned.
    where: str, optional
        any additional custom sql filtering you wish to apply. For example if `where` = 'temperature > 100',
        only data values where temperature is greater than 100 will be returned.
        None by default. When `where` = None, no additional filtering is applied to the data.
    verbose: bool, optional
        If `verbose` = True, the function prints the sql query used to retrieve the data.
        False by default.
    
        
    Raises
    ------
    TypeError
        If `table` is not a string
    """
    
    if type(table) != str:
        raise TypeError('`table` argument must be a string.')
    
    data = None
    try:
        cur = conn.cursor()
        if fields is None:
            sql_query = 'SELECT * FROM '
        elif type(fields) == list:
            sql_query = 'SELECT ' + ', '.join(fields) + ' FROM '
        else:
            sql_query = 'SELECT ' + fields + ' FROM '
        sql_query += table
        if sensor is not None:
            if type(sensor) == list:
                sql_query += ' WHERE sensor in (' + str(sensor)[1:len(str(sensor))-1] + ')'
            else:
                sql_query += ' WHERE sensor == ' + str(sensor)
        if time is not None:
            if 'WHERE' in sql_query:
                sql_query += ' AND time >= \'' + time[0] + '\' AND time <= \'' + time[1] + '\''
            else:
                sql_query += ' WHERE time >= \'' + time[0] + '\' AND time <= ' + time[1] + '\''
        if where is not None:
            if 'WHERE' in sql_query:
                sql_query += ' AND ' + where
            else:
                sql_query += ' WHERE ' + where
        sql_query += ';'
        cur.execute(sql_query)
        data = cur.fetchall()
    except Error as e:
        print(e)      
    if verbose:
        print(sql_query)
    return data

        
def update_data(conn, table, values, where, fields = None, verbose = False):
    """changes certain values in a table in the database
    
    If `fields` argument is not passed, all columns will be changed, 
    and the `values` agrument must be an entire row of data
    
    Parameters
    ----------
    conn: sqlite3.connect
        a connection to a sqlite3 database 
    table: str
        the name of a table in the database
    values: list
        a list of new values you want to change certain values to in the table.
        The length of `values` must be equal to the number of columns in table if `fields` is not supplied.
        If `fields` is supplied, the length of `values` must be equal to the length of `fields`.
        They may both be a single value, if you only wish to change the value of a single column. 
    where: str
        a string that specifies which rows to change the data in.
        For example if `where` = 'time == '2021-04-19 15:33:50' and sensor == 1',
        the data will be changed for the reading of sensor 1 at 15:33:50 on 2021-04-19
    fields: list[str], optional
        a list of columns you wish to change the values of.
        None by default. When `fields` = None, every column will be changed.
        `fields` can also be a single string object if you wish to change only one column from the table.
    verbose: bool, optional
        If `verbose` = True, the function prints the sql query used to create the new feature.
        False by default.
        
    Raises
    ------
    TypeError
        If `table` is not a string
    ValueError
        If the length of `values` does not conform to the length of `fields`
    """
    
    if type(table) != str:
        raise TypeError('`table` argument must be a string.')
    if fields is None:
        if type(values) != list or len(values) != len(get_fields(conn, table)):
            raise ValueError('If `fields` argument is not supplied, `values` argument must be a list of length equal to the number of columns in the table.')
    else:
        if type(values) == list and type(fields) == list:
            if len(values) != len(fields):
                raise ValueError('If `fields` argument is supplied, lengths of `fields` and `values` must be equal.')
        else:
            if (type(values) == list and type(fields) != list) or (type(values) != list and type(fields) == list):
                raise ValueError('If one of `fields` or `values` is a list, the other must also be a list')
    sql_update = 'UPDATE ' + table + ' SET'
    if type(fields) == list:
        for x,y in zip(fields, values):
            if type(y) == str:
                sql_update += str(' ' + x + ' = \'' + str(y) + '\',')
            else:
                sql_update += str(' ' + x + ' = ' + str(y) + ',')
        sql_update = sql_update[:len(sql_update)-1]
    else:
        if type(values) == str:
            sql_update += str(' ' + fields + ' = ' + '\''+str(values)+'\'')
        else:
            sql_update += str(' ' + fields + ' = ' +str(values))
    sql_update += ' WHERE ' + where + ';'
    try:
        cur = conn.cursor()
        cur.execute(sql_update)
        conn.commit()
    except Error as e:
        print(e)  
    if verbose:
        print(sql_update)


def new_feature(conn, table, sensor, field, operator, name, verbose = False):
    """create a new table which contains a new feature calculated from measurements from two different sensors.
    The sensors can be on the same machinery or different machinery. The measurements will be joined by timestamp.
    
    Parameters
    ----------
    conn: sqlite3.connect
        a connection to a sqlite3 database 
    table: str
        the name of a table in the database or a list of two tables
    sensor: list[int]
        a list of length two that contains the sensor ids of the two sensors you want to use to create the new feature.
        Can possibly be the same sensor, but still has to be a list, such as [1,1]
    field: str
        The name of the column from which you want to create the new feature.
        Can be a list of two strings if you want to create a new feature that is a combination of two different measurements.
    operator: str
        mathematical symbol you wish to use to calculate the new feature 
        such as '+', '-', '/', or '*'.
    name: str
        The name you want to give to the new table and the new feature.
    verbose: bool, optional
        If `verbose` = True, the function prints the sql query used to retrieve the data.
        False by default.
        
    Raises
    ------
    TypeError
        If `table` is not a string
        If `operator` is not a string
        If `name` is not a string
    ValueError
        If `sensor` is not a list of length 2.
    """
    if type(sensor) != list or len(sensor) != 2:
        raise ValueError('sensor argument must be a list of length 2')
    if type(table) != str:
        raise TypeError('`table` argument must be a string.')
    if type(operator) != str:
        raise TypeError('`operator` argument must be a string.')        
    if type(name) != str:
        raise TypeError('`name` argument must be a string.')
        
    sql_feature = 'SELECT d1.time, d1.' + field + operator + 'd2.' + field + ' as ' + name + ' FROM (SELECT * FROM '
    if type(table) == list:
        sql_feature += table[0] + ' WHERE sensor == ' + str(sensor[0]) + ') as d1 JOIN (SELECT * FROM ' + table[1] + ' WHERE sensor == ' + str(sensor[1]) + ') as d2 '
    else:
        sql_feature += table + ' WHERE sensor == ' + str(sensor[0]) + ') as d1 JOIN (SELECT * FROM ' + table + ' WHERE sensor == ' + str(sensor[1]) + ') as d2 '
    sql_feature += 'ON d1.time = d2.time'
    try:
        create_table(conn, name, ['time',name])
        cur = conn.cursor()
        cur.execute(sql_feature)
        for row in cur.fetchall():
            insert_data(conn, name, row)
        conn.commit()
    except Error as e:
        print(e)  
    if verbose:
        print(sql_feature)
        
        

def main():
    conn = create_connection("iot.db")
    import_json(conn, "data.json")
    return conn
    
if __name__ == "__main__":
    conn = main()

# Examples:

In [2]:
#print the names of the tables in the database.
print(get_tables(conn))

#print the names of the columns of the tables.
print(get_fields(conn, 'valve'))
print(get_fields(conn, 'oven'))

['valve', 'oven']
['time', 'devicetype', 'sensor', 'temperature', 'pressure', 'vibration']
['time', 'devicetype', 'sensor', 'temperature', 'humidity']


In [3]:
#print all of the data from the oven data set
for row in get_data(conn, 'oven'):
    print(row)

('2021-04-19 15:33:50', 'oven', 1, 109.0441, 22.3144)
('2021-04-19 15:33:50', 'oven', 2, 95.0998, 28.4975)
('2021-04-19 15:33:50', 'oven', 3, 106.0025, 27.6463)
('2021-04-19 15:34:00', 'oven', 2, 107.3204, 30.8018)
('2021-04-19 15:34:10', 'oven', 2, 103.2723, 37.2119)
('2021-04-19 15:34:20', 'oven', 1, 79.4583, 25.1541)
('2021-04-19 15:34:20', 'oven', 2, 106.4702, 37.9055)
('2021-04-19 15:34:30', 'oven', 2, 96.2432, 32.0144)
('2021-04-19 15:34:40', 'oven', 2, 103.837, 31.1623)
('2021-04-19 15:34:50', 'oven', 1, 91.762, 20.7965)
('2021-04-19 15:34:50', 'oven', 2, 104.956, 26.1649)
('2021-04-19 15:34:50', 'oven', 3, 106.5797, 38.5498)
('2021-04-19 15:35:00', 'oven', 2, 97.1462, 33.4181)
('2021-04-19 15:35:10', 'oven', 2, 91.6005, 37.1309)
('2021-04-19 15:35:20', 'oven', 1, 100.8957, 25.876)
('2021-04-19 15:35:20', 'oven', 2, 94.4203, 35.9877)
('2021-04-19 15:35:30', 'oven', 2, 96.8015, 21.2551)
('2021-04-19 15:35:40', 'oven', 2, 109.2717, 31.2631)
('2021-04-19 15:35:50', 'oven', 1, 48.08

In [4]:
#print only the measurments from sensor 1
for row in get_data(conn, 'oven', sensor = 1):
    print(row)

('2021-04-19 15:33:50', 'oven', 1, 109.0441, 22.3144)
('2021-04-19 15:34:20', 'oven', 1, 79.4583, 25.1541)
('2021-04-19 15:34:50', 'oven', 1, 91.762, 20.7965)
('2021-04-19 15:35:20', 'oven', 1, 100.8957, 25.876)
('2021-04-19 15:35:50', 'oven', 1, 48.0859, 24.8943)
('2021-04-19 15:36:20', 'oven', 1, 120.8344, 20.2867)
('2021-04-19 15:36:50', 'oven', 1, 99.0771, 24.1381)
('2021-04-19 15:37:20', 'oven', 1, 90.5516, 27.3245)
('2021-04-19 15:37:50', 'oven', 1, 111.8005, 23.0301)
('2021-04-19 15:38:20', 'oven', 1, 105.8424, 21.2801)
('2021-04-19 15:38:50', 'oven', 1, 108.7485, 28.3054)


In [5]:
#print only the measurements from sensor 1 or 2 
#that were taken after 3:35 pm and where the temperature was at least 98 degrees
for row in get_data(conn, 'oven', sensor = [1,2], time = ['2021-04-19 15:35:00','3000-01-01 00:00:00'], where = 'temperature >= 98'):
    print(row)

('2021-04-19 15:35:20', 'oven', 1, 100.8957, 25.876)
('2021-04-19 15:35:40', 'oven', 2, 109.2717, 31.2631)
('2021-04-19 15:36:00', 'oven', 2, 105.8488, 34.202)
('2021-04-19 15:36:20', 'oven', 1, 120.8344, 20.2867)
('2021-04-19 15:36:20', 'oven', 2, 101.6294, 39.0691)
('2021-04-19 15:36:50', 'oven', 1, 99.0771, 24.1381)
('2021-04-19 15:37:40', 'oven', 2, 113.7173, 35.9687)
('2021-04-19 15:37:50', 'oven', 1, 111.8005, 23.0301)
('2021-04-19 15:38:20', 'oven', 1, 105.8424, 21.2801)
('2021-04-19 15:38:20', 'oven', 2, 99.0738, 37.13)
('2021-04-19 15:38:40', 'oven', 2, 104.3849, 23.6648)
('2021-04-19 15:38:50', 'oven', 1, 108.7485, 28.3054)
('2021-04-19 15:38:50', 'oven', 2, 104.9351, 24.6487)


In [6]:
#print all pressure measurements from sensor 2 from the valve table
[x[0] for x in get_data(conn, 'valve', sensor = 2, fields = 'pressure')]

[0.6975,
 0.5829,
 -0.731,
 -1.7987,
 -2.7045,
 -1.4401,
 0.7774,
 2.2034,
 0.4918,
 1.295,
 1.1097,
 0.2767,
 -0.3732,
 -0.2038,
 1.2972,
 2.5206,
 1.764,
 0.3483,
 -1.0168,
 -0.5012,
 -0.3693,
 -0.1016,
 -1.0719,
 0.5761,
 1.8481,
 1.4094,
 0.2581,
 -0.8967,
 -1.9331,
 -0.7635,
 0.57]

In [7]:
#change the first temperature reading for sensor 1 from the oven table to 999 degrees
update_data(conn, 'oven', 999, where = 'sensor == 1 AND time = \'2021-04-19 15:33:50\'', fields = 'temperature')

get_data(conn, 'oven', sensor = 1)

[('2021-04-19 15:33:50', 'oven', 1, 999, 22.3144),
 ('2021-04-19 15:34:20', 'oven', 1, 79.4583, 25.1541),
 ('2021-04-19 15:34:50', 'oven', 1, 91.762, 20.7965),
 ('2021-04-19 15:35:20', 'oven', 1, 100.8957, 25.876),
 ('2021-04-19 15:35:50', 'oven', 1, 48.0859, 24.8943),
 ('2021-04-19 15:36:20', 'oven', 1, 120.8344, 20.2867),
 ('2021-04-19 15:36:50', 'oven', 1, 99.0771, 24.1381),
 ('2021-04-19 15:37:20', 'oven', 1, 90.5516, 27.3245),
 ('2021-04-19 15:37:50', 'oven', 1, 111.8005, 23.0301),
 ('2021-04-19 15:38:20', 'oven', 1, 105.8424, 21.2801),
 ('2021-04-19 15:38:50', 'oven', 1, 108.7485, 28.3054)]

In [8]:
#create a new feature that's the ratio of the temperature in sensor 1 and the 
#temperature in sensor 2 in the valve data set (and print the SQL command)
new_feature(conn, 'valve', sensor = [1,2], field = 'temperature', operator = '/', name = 'valve_temp_ratio', verbose = True)

SELECT d1.time, d1.temperature/d2.temperature as valve_temp_ratio FROM (SELECT * FROM valve WHERE sensor == 1) as d1 JOIN (SELECT * FROM valve WHERE sensor == 2) as d2 ON d1.time = d2.time


In [9]:
#now let's make sure the new table exists
'valve_temp_ratio' in get_tables(conn)

True

In [13]:
#and let's print it out
for row in get_data(conn, 'valve_temp_ratio'):
    print(row)

('2021-04-19 15:33:50', 0.9183526546687055)
('2021-04-19 15:34:50', 1.0321300524170676)
('2021-04-19 15:35:50', 1.2443659967819296)
('2021-04-19 15:36:50', 0.8661847302833557)
('2021-04-19 15:37:50', 1.0483834945009838)
('2021-04-19 15:38:50', 0.994181695371835)


# Discussion

### pros and cons

Pros:
    * Database is persistent. It exists outside of the connection.
    * You can import multiple json files, and if there is data that has the same table name, the data is added to the appropriate table rather than creating a new table.
    * Relatively easy to use functions. You don't need to know SQL to use the functions I wrote.
    * Pretty flexible. You can retrieve the data you want and save them as Python objects. 
Cons:
    * Functions are pretty limited as far as SQL manipulations that you can do. Particularly the new_feature function. You can only perform simple arithmatic operations, and it can only be done on two sensors at a time. However, if you wanted to do something like the mean of two measurements, you could simply use the function so compute the sum, and then use another function to divide all the numbers by two.
    * The system is very specific about the formatting of the json data. I'm almost positive the fake data I came up with is not formatted in the way the real data is formatted.
    * If two measurements take place in the same second, they are considered 'paired' when coming up with a new feature. In real life, maybe the time measurements are more precise than that. Or maybe it would be better to be able to specify a certain time window, and if any measurements are within that window of eachother, they would be averaged, and then merged with measurements over the same time period from other sensors.
    * I didn't consider the handling of missing values. I'm used to just leaving NA's intact. In my master's program I was taught that you need a very compelling reason to remove or impute missing values. I did consider how to handle duplicate rows (it simply doesn't append them if they are a duplicate). Maybe it would be better to ask the user "This row is a duplicate, should it be ignored? Y/N." Or maybe it could have been an extra argument skipDuplicates = True. 
    * In addition to missing and duplicated values, another thing my system doesn't handle is obviously wrong values. I think if I had more time, I would like to have a system that defines acceptable values for each type of variable. If a value outside of the acceptable range is found, the program can ask the user what they'd like to do. Or it could be a global setting like by default, make all unacceptable values NA, and keep a log of which tables had unacceptable values, which rows they were in, what the values were. 